In [ ]:
!nvcc --version
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin

# obliczanie równoległe liczby PI z wzoru Leibniz-a

# Jak działa kod?
Kod jest podzielony na dwie części, jedną część wykonuje główny proces (ranga 0), a drugą część wykonują pozostałe procesy.

Główny proces odbiera wyniki obliczeń od innych procesów za pomocą MPI_Recv(), dodaje je i wyświetla wynik końcowy.

Pozostałe procesy obliczają wartość Pi za pomocą formuły Leibniza i wysyłają wyniki z powrotem do głównego procesu za pomocą MPI_Send(). Także mierzą swój własny czas wykonania i wysyłają go z powrotem do głównego procesu. Główny proces następnie dodaje czasy wykonania ze wszystkich procesów i wyświetla całkowity czas wykonania.

Funkcja potegowanie() oblicza potęgę liczby. Reszta kodu inicjalizuje MPI, pobiera rangę i rozmiar procesów i implementuje komunikację między procesami.

In [ ]:
%%sh
cat > pi-mpi.c << EOF

#include <mpi.h>
#include <stdio.h>
#include <time.h>
 
// Funkcja do obliczania potęgi
double potegowanie(double a , int b )
{
      double result=1.0;
      for (int i = 1; i <= b; i++)
      {
          result *= a;
      }
      return result; 
}

int main(int argc, char **argv)
{
    int rank; // Numer procesu
    int size; // Liczba procesów
    int tag = 1; // Zmienna oznaczająca tag dla każdej wiadomości przesłanej pomiędzy procesami
    MPI_Status status; // Zmienna przechowująca informację o statusie wysłania lub odebrania wiadomości
    MPI_Init(&argc, &argv); // Inicjalizacja MPI
    MPI_Comm_rank(MPI_COMM_WORLD, &rank); // Pobieranie numeru procesu
    MPI_Comm_size(MPI_COMM_WORLD, &size); // Pobieranie liczby procesów
    clock_t t; // Zmienna przechowująca czas początkowy dla danego procesu
    double time_taken; // Zmienna przechowująca czas wykonania danego procesu
     // Jeśli jest to proces o numerze 0
    if(rank == 0)
    {
      double lpi = 0 ,x = 0,total_time =0,t=0; // Zmienne przechowujące wynik dla każdego procesu oraz łączny czas wykonania
        // Pętla, w której proces 0 odbiera wyniki od pozostałych procesów
      for(int i=1; i<size; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status); // Odebranie wyniku dla danego procesu
        lpi +=x; // Dodanie wyniku do łącznego wyniku
        MPI_Recv(&t, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status); // Odebranie informacji o czasie wykonania dla danego procesu
        total_time+=t; // Dodanie czasu do łącznego czasu wykonywania
      }
      printf("Wartość liczby pi = %f \n", 4 * lpi);
      printf("Całkowity czas wykonywania = %f \n", total_time);
    }
     // warunek sprawdzający czy proces jest głównym procesem. Jeśli tak, nie wykonuje dalszych operacji.
    if(rank != 0)
    {
      t = clock(); // zmienna "t" jest ustawiana na bieżący czas systemowy, aby móc potem obliczyć czas wykonywania.
      // Obliczanie wyniku dla danego procesu
      double x = potegowanie( -1, rank - 1) / ( 2 * rank - 1);  // obliczenie wartości x jako wyniku wywołania funkcji "potegowanie()" dzielonego przez (2 * rank - 1).
      printf("Wynik procesu %d = %f \t" , rank , x); // wypisanie na ekran wyniku dla danego procesu.
      // wysłanie wyniku do głównego procesu 0 za pomocą funkcji MPI_Send()
      MPI_Send(&x, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      t = clock() - t; // obliczenie czasu wykonywania poprzez odejmowanie bieżącego czasu od zmiennej "t".
      time_taken = ((double)t)/CLOCKS_PER_SEC;
      // Wysyłanie informacji o czasie wykonania do procesu 0
      MPI_Send(&time_taken, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
      printf("Czas wykonywania sie procesu %d = %f\n",rank,time_taken);
    }
    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 60 --allow-run-as-root a.out

Ten kod jest implementacją algorytmu wyznaczania całki numerycznej funkcji kwadratowej za pomocą biblioteki MPI (Message Passing Interface). Zmienna "rank" przechowuje numer procesu, a zmienna "size" przechowuje liczbę procesów. Następnie, przy użyciu funkcji MPI_Init(), MPI_Comm_rank() i MPI_Comm_size() inicjalizowana jest biblioteka MPI i pobierane są numery procesów i liczba procesów. Zmienna "h" jest krokiem calkowania, a proces o numerze 0 jest głównym procesem i odbiera wyniki od pozostałych procesów. Pozostałe procesy wyznaczają wartość funkcji i wysyłają ją do głównego procesu, a następnie wyświetlają swój czas wykonania. Po zakończeniu wszystkich obliczeń, biblioteka MPI jest zakończona przy użyciu funkcji MPI_Finalize().

In [ ]:
%%sh
cat > pi-mpi.c << EOF
#include <mpi.h>
#include <stdio.h>
#include <time.h>
 
// Funkcja, którą calkujemy
double funkcja(double x)
{
  return (x * x + 2 * x + 3*x);
}

int main(int argc, char **argv)
{
  // Inicjalizacja zmiennych potrzebnych do obsługi MPI
    int rank; // Numer procesu
    int size; // Liczba procesów
    int tag = 1; // Zmienna oznaczająca tag dla każdej wiadomości przesłanej pomiędzy procesami
    
    MPI_Status status; // Zmienna przechowująca status odebrania wiadomości
    MPI_Init(&argc, &argv); // Inicjalizacja MPI
    MPI_Comm_rank(MPI_COMM_WORLD, &rank); // Pobranie numeru procesu
    MPI_Comm_size(MPI_COMM_WORLD, &size); // Pobranie liczby procesów
    clock_t t; // Zmienna do mierzenia czasu
    double time_taken; // Zmienna do przechowywania czasu wykonania procesu
    // Zmienne do przechowywania liczby procesów i numeru procesu w formacie double
    double numproc = size, nmbproc = rank;
    // Granice calkowania
    double lower = 1,upper = 4;
    // Krok calkowania
    double h = (upper - lower) / numproc;
    // Proces o numerze 0 jest głównym procesem i odbiera wyniki od pozostałych procesów
    if(rank == 0)
    {
      double calka = 0 ,x=0;
      // Odbieranie wyników od pozostałych procesów
      for(int i=1; i<size; i++)
      {
        MPI_Recv(&x, 1, MPI_DOUBLE, i, tag, MPI_COMM_WORLD, &status);
        calka +=x;
      }
      // Wypisanie wyniku
      printf("Wartosc dolnej granicy calki = %f \n",funkcja(lower)/2 * h);
      printf("Wartosc gornej granicy calki = %f \n",funkcja(upper)/2 * h);

      printf("Wartosc calki = %1f \n", h * (funkcja(lower)/2 + calka 
        + funkcja(upper)/2));
    }
    // Sprawdź czy numer procesu jest różny od 0 jeśli tak wywołaj funkcje
    if(rank != 0)
    {
        t = clock(); 
        double xi,wynik;
        nmbproc = rank; // numer procesu
        xi = lower + (nmbproc/numproc)  
            * (upper - lower);
        wynik = funkcja(xi);
        printf("Wynik  %d = %f \t", rank,wynik*h);
        // Wynik jest wysyłany do procesu 0
        MPI_Send(&wynik, 1, MPI_DOUBLE, 0, tag, MPI_COMM_WORLD);
        t = clock() - t;
        time_taken = ((double)t)/CLOCKS_PER_SEC;
        printf("Czas wykonywania:  %d = %f\n",rank,time_taken);
    }

    MPI_Finalize();
}
EOF
mpicc pi-mpi.c && mpirun -n 100 --allow-run-as-root a.out